In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# For connect to google sheet
import gspread

/home/jahn/.asdf/installs/python/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
print("hello")

hello
